In [1]:
import numpy as np

In [2]:
size = 4
B = np.array([
    1,1,0,0,
    0,0,1,1,
    1,1,0,0,
    0,0,1,0
]).reshape((size, size))

In [3]:
B

array([[1, 1, 0, 0],
       [0, 0, 1, 1],
       [1, 1, 0, 0],
       [0, 0, 1, 0]])

In [4]:
def log_matrix_deg(a, n):
    if n == 0:
        return np.eye(a.shape[0])
    if n == 1:
        return a

    if n % 2 == 0:
        return log_matrix_deg(np.dot(a, a), int(n / 2))
    else:
        return np.dot(a, log_matrix_deg(np.dot(a, a), int((n - 1) / 2)))

In [8]:
n = 3
np.sum(np.dot(log_matrix_deg(B, n - 2), np.ones(size)), dtype=int)

7

In [ ]:
import numpy as np


def log_matrix_deg(a, n):
    if n == 0:
        return np.eye(a.shape[0])
    if n == 1:
        return a

    if n % 2 == 0:
        return log_matrix_deg(np.dot(a, a), int(n / 2))
    else:
        return np.dot(a, log_matrix_deg(np.dot(a, a), int((n - 1) / 2)))
    
    
if __name__ == '__main__':
    size = 4
    B = np.array([
        1,1,0,0,
        0,0,1,1,
        1,1,0,0,
        0,0,1,0
    ]).reshape((size, size))
    n = int(input().stripe())
    print(np.sum(np.dot(log_matrix_deg(B, n - 2), np.ones(size)), dtype=int))

In [57]:
size = 10
A = np.array([1 if np.abs(i - j) <= 1 else 0 for i in range(size) for j in range(size)]).reshape((size, size))

In [58]:
A 

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]])

In [59]:
a_deg(A, 2)

array([[2, 2, 1, 0, 0, 0, 0, 0, 0, 0],
       [2, 3, 2, 1, 0, 0, 0, 0, 0, 0],
       [1, 2, 3, 2, 1, 0, 0, 0, 0, 0],
       [0, 1, 2, 3, 2, 1, 0, 0, 0, 0],
       [0, 0, 1, 2, 3, 2, 1, 0, 0, 0],
       [0, 0, 0, 1, 2, 3, 2, 1, 0, 0],
       [0, 0, 0, 0, 1, 2, 3, 2, 1, 0],
       [0, 0, 0, 0, 0, 1, 2, 3, 2, 1],
       [0, 0, 0, 0, 0, 0, 1, 2, 3, 2],
       [0, 0, 0, 0, 0, 0, 0, 1, 2, 2]])

In [17]:
import numpy as np
from sys import stdin


def dot_n3(a, b):
    assert a.shape[1] == b.shape[0]
    c = np.zeros((a.shape[0], b.shape[1]), dtype=int)
    for i in range(a.shape[0]):
        for k in range(b.shape[1]):
            c[i, k] = np.sum(a[i, j] * b[j, k] for j in range(a.shape[1])) % 9
    return c


def get_deg(size):
    return int(2**np.ceil(np.log(size) / np.log(2)))


def size2deg(a):
    assert a.shape[0] == a.shape[1]
    size = get_deg(a.shape[0])
    matrix = np.zeros((size, size), dtype=int)
    matrix[:a.shape[0], :a.shape[0]] = a
    return matrix


def strassen(a, b):
    assert a.shape[0] == a.shape[1] and a.shape[0] == get_deg(a.shape[0])
    assert b.shape[0] == b.shape[1] and b.shape[0] == get_deg(b.shape[0])
    assert a.shape[0] == b.shape[0]
    size = int(a.shape[0] / 2)
    if size <= 8:
        return dot_n3(a, b)
    
    # a submatrixes
    a11 = a[:size, :size]
    a12 = a[:size, size:]
    a21 = a[size:, :size]
    a22 = a[size:, size:]
#     print(a11, a12, a21, a22, sep='\n')
    
    # b submatrixes
    b11 = b[:size, :size]
    b12 = b[:size, size:]
    b21 = b[size:, :size]
    b22 = b[size:, size:]
#     print(b11, b12, b21, b22, sep='\n')

    # help matrix
    p1 = strassen(a11 + a22, b11 + b22)
    p2 = strassen(a21 + a22, b11)
    p3 = strassen(a11, b12 - b22)
    p4 = strassen(a22, b21 - b11)
    p5 = strassen(a11 + a12, b22)
    p6 = strassen(a21 - a11, b11 + b12)
    p7 = strassen(a12 - a22, b21 + b22)
    
    # answer
    c11 = p1 + p4 - p5 + p7
    c12 = p3 + p5
    c21 = p2 + p4
    c22 = p1 - p2 + p3 + p6
    
    c = np.concatenate([
        np.concatenate([c11, c12], axis=1),
        np.concatenate([c21, c22], axis=1)
    ], axis=0)
    return c


def strip_matrix(a, size):
    return a[:size, :size]


def fast_matrix_deg(a, n):
    if n == 0:
        return np.eye(a.shape[0])
    if n == 1:
        return a

    if n % 2 == 0:
        return fast_matrix_deg(strassen(a, a), int(n / 2))
    else:
        return strassen(a, fast_matrix_deg(strassen(a, a), int((n - 1) / 2)))


def a_deg(a, deg):
    assert a.shape[0] == a.shape[1]
    n = a.shape[0]
    a_wide = size2deg(a)
    a_deg = fast_matrix_deg(a_wide, deg)
    a_strip = strip_matrix(a_deg, n)
    return a_strip

In [60]:
np.sum(np.dot(a_deg(A, 1), np.array([1 for i in range(size)]))[1:])

26

In [41]:
size = 4
B = np.array([
    1,1,0,0,
    0,0,1,1,
    1,1,0,0,
    0,0,1,0
]).reshape((size, size))

In [51]:
n = 1
np.sum(np.dot(a_deg(B, n - 2), np.ones(size)), dtype=int)

RecursionError: maximum recursion depth exceeded in comparison

In [26]:
np.dot(a_deg(A, 2), np.array([1 for i in range(size)]))

array([2, 3, 4, 4, 4, 4, 4, 4, 3, 2])

In [28]:
np.dot(a_deg(A, 2), np.array([1 for i in range(size)]))

array([2, 3, 4, 4, 4, 4, 4, 4, 3, 2])

In [22]:
np.array([0 if i == 0 else 1 for i in range(size)])

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [3]:
size_i = 3
size_j = 4
size_k = 2

a = np.random.randint(0, 8, size=size_i*size_j).reshape((size_i, size_j))
b = np.random.randint(0, 8, size=size_j*size_k).reshape((size_j, size_k))

In [4]:
print(a)
print(b)

[[1 3 2 1]
 [6 7 5 3]
 [6 4 2 5]]
[[1 2]
 [6 3]
 [0 1]
 [1 5]]


In [5]:
print(np.dot(a, b))

[[20 18]
 [51 53]
 [35 51]]


In [17]:
def dot_n3(a, b):
    assert a.shape[1] == b.shape[0]
    c = np.zeros((a.shape[0], b.shape[1]), dtype=int)
    for i in range(a.shape[0]):
        for k in range(b.shape[1]):
            c[i, k] = np.sum(a[i, j] * b[j, k] for j in range(a.shape[1]))
    return c

In [18]:
print(dot_n3(a, b))

[[20 18]
 [51 53]
 [35 51]]


In [70]:
def get_deg(size):
    return int(2**np.ceil(np.log(size)/np.log(2)))

In [71]:
get_deg(100)

128

In [44]:
def size2deg(a):
    assert a.shape[0] == a.shape[1]
    size = get_deg(a.shape[0])
    matrix = np.zeros((size, size), dtype=int)
    matrix[:a.shape[0], :a.shape[0]] = a
    return matrix

In [33]:
d = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))

In [34]:
print(d)

[[0 7 1]
 [5 3 1]
 [6 7 4]]


In [35]:
size2deg(d)

array([[0, 7, 1, 0],
       [5, 3, 1, 0],
       [6, 7, 4, 0],
       [0, 0, 0, 0]])

In [39]:
a = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))
b = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))
aa = size2deg(a)
bb = size2deg(b)

In [40]:
print(a)
print(b)

[[6 6 3]
 [7 6 0]
 [3 7 4]]
[[4 1 2]
 [3 5 0]
 [4 4 2]]


In [41]:
print(aa)
print(bb)

[[6 6 3 0]
 [7 6 0 0]
 [3 7 4 0]
 [0 0 0 0]]
[[4 1 2 0]
 [3 5 0 0]
 [4 4 2 0]
 [0 0 0 0]]


In [57]:
np.dot(aa, bb)

array([[54, 48, 18,  0],
       [46, 37, 14,  0],
       [49, 54, 14,  0],
       [ 0,  0,  0,  0]])

In [62]:
def strassen(a, b):
    assert a.shape[0] == a.shape[1] and a.shape[0] == get_deg(a.shape[0])
    assert b.shape[0] == b.shape[1] and b.shape[0] == get_deg(b.shape[0])
    assert a.shape[0] == b.shape[0]
    size = int(a.shape[0]/2)
    if size <= 8:
        return dot_n3(a, b)
    
    # a submatrixes
    a11 = a[:size, :size]
    a12 = a[:size, size:]
    a21 = a[size:, :size]
    a22 = a[size:, size:]
#     print(a11, a12, a21, a22, sep='\n')
    
    # b submatrixes
    b11 = b[:size, :size]
    b12 = b[:size, size:]
    b21 = b[size:, :size]
    b22 = b[size:, size:]
#     print(b11, b12, b21, b22, sep='\n')

    # help matrix
    p1 = strassen(a11 + a22, b11 + b22)
    p2 = strassen(a21 + a22, b11)
    p3 = strassen(a11, b12 - b22)
    p4 = strassen(a22, b21 - b11)
    p5 = strassen(a11 + a12, b22)
    p6 = strassen(a21 - a11, b11 + b12)
    p7 = strassen(a12 - a22, b21 + b22)
    
    # answer
    c11 = p1 + p4 - p5 + p7
    c12 = p3 + p5
    c21 = p2 + p4
    c22 = p1 - p2 + p3 + p6
    
    c = np.concatenate([
        np.concatenate([c11, c12], axis=1),
        np.concatenate([c21, c22], axis=1)
    ], axis=0)
    return c

strassen test

In [77]:
size_i = 300
a = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))
b = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))
aa = size2deg(a)
bb = size2deg(b)
np.array_equal(strassen(aa, bb), np.dot(aa, bb))

True

In [78]:
%%time
r = np.dot(aa, bb)

CPU times: user 274 ms, sys: 3.76 ms, total: 277 ms
Wall time: 275 ms


In [79]:
%%time
r = strassen(aa, bb)

CPU times: user 37.4 s, sys: 139 ms, total: 37.6 s
Wall time: 37.7 s


In [80]:
%%time
r = dot_n3(aa, bb)

CPU times: user 1min 4s, sys: 276 ms, total: 1min 4s
Wall time: 1min 5s


In [82]:
def strip_matrix(a, size):
    return a[:size, :size]

In [85]:
size_i = 3
a = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))
b = np.random.randint(0, 8, size=size_i*size_i).reshape((size_i, size_i))
aa = size2deg(a)
bb = size2deg(b)

In [86]:
aa

array([[1, 0, 6, 0],
       [6, 4, 4, 0],
       [4, 5, 2, 0],
       [0, 0, 0, 0]])

In [87]:
strip_matrix(aa, a.shape[0])

array([[1, 0, 6],
       [6, 4, 4],
       [4, 5, 2]])

In [88]:
size_i = 5
a = np.random.randint(-20, 20, size=size_i*size_i).reshape((size_i, size_i))

In [89]:
a

array([[ 16, -20, -10,  11, -11],
       [  5, -19,   3,  -7, -10],
       [  0,  16,  18, -11, -16],
       [-14, -17, -20,  13,  17],
       [ -4, -15,  -3,  -7, -13]])

In [93]:
def matix2Z(a, Z=9):
    shape = a.shape
    return np.array(list(map(
        lambda x: x%Z, a
    ))).reshape(shape)

In [94]:
matix2Z(a, Z=9)

array([[7, 7, 8, 2, 7],
       [5, 8, 3, 2, 8],
       [0, 7, 0, 7, 2],
       [4, 1, 7, 4, 8],
       [5, 3, 6, 2, 5]])

In [96]:
def fast_matrix_deg(a, n):
    if n == 0:
        return np.eye(a.shape[0])
    if n == 1:
        return a
    
    if n % 2 == 0:
        return fast_matrix_deg(strassen(a, a), int(n/2))
    else:
        return strassen(a, fast_matrix_deg(strassen(a, a), int((n - 1)/2)))
    

def do_task(a):
    assert a.shape[0] == a.shape[1]
    n = a.shape[0]
    a_wide = size2deg(a)
    a_deg = fast_matrix_deg(a_wide, n)
    a_strip = strip_matrix(a_deg, n)
    a_z9 = matix2Z(a_strip, Z=9)
    return a_z9

In [97]:
size_i = 5
a = np.random.randint(-20, 20, size=size_i*size_i).reshape((size_i, size_i))

In [98]:
a

array([[-17,  -4, -18, -12,  -1],
       [  8, -16,  -9,  -1,   2],
       [ -4,  -9,  -6, -14, -15],
       [ 10,   6,   9,  -1, -13],
       [ 10, -10,  14,   5,  -8]])

In [100]:
res = a
for i in range(5-1):
    res = np.dot(res, a)

In [102]:
matix2Z(res, Z=9)

array([[0, 6, 6, 6, 4],
       [3, 2, 0, 7, 7],
       [3, 1, 0, 8, 0],
       [1, 2, 3, 0, 1],
       [5, 2, 4, 0, 8]])

In [99]:
do_task(a)

array([[0, 6, 6, 6, 4],
       [3, 2, 0, 7, 7],
       [3, 1, 0, 8, 0],
       [1, 2, 3, 0, 1],
       [5, 2, 4, 0, 8]])

In [ ]:
from sys import stdin

def read_matrix():
    a = list(map(int, stdin.readline().rstrip().split()))
    n = len(a)
    for _ in range(n - 1):
        a.extend(list(map(int, stdin.readline().rstrip().split())))
        
    return np.array(a, dtype=int).reshape((n, n))




In [53]:
np.concatenate([aa, aa], axis=0)

array([[6, 6, 3, 0],
       [7, 6, 0, 0],
       [3, 7, 4, 0],
       [0, 0, 0, 0],
       [6, 6, 3, 0],
       [7, 6, 0, 0],
       [3, 7, 4, 0],
       [0, 0, 0, 0]])

In [50]:
sh(aa, bb)

[[6 6]
 [7 6]]
[[3 0]
 [0 0]]
[[3 7]
 [0 0]]
[[4 0]
 [0 0]]
[[4 1]
 [3 5]]
[[2 0]
 [0 0]]
[[4 4]
 [0 0]]
[[2 0]
 [0 0]]


In [37]:
print(a)
print(b)

[[5 6 1]
 [1 0 7]
 [6 4 0]]
[[2 0 6]
 [5 5 7]
 [6 1 3]]


In [164]:
import numpy as np
from sys import stdin

def dot_n3(a, b):
    assert a.shape[1] == b.shape[0]
    c = np.zeros((a.shape[0], b.shape[1]), dtype=int)
    for i in range(a.shape[0]):
        for k in range(b.shape[1]):
            c[i, k] = np.sum(a[i, j] * b[j, k] for j in range(a.shape[1])) % 9
    return c


def get_deg(size):
    return int(2**np.ceil(np.log(size)/np.log(2)))


def size2deg(a):
    assert a.shape[0] == a.shape[1]
    size = get_deg(a.shape[0])
    matrix = np.zeros((size, size), dtype=int)
    matrix[:a.shape[0], :a.shape[0]] = a
    return matrix


def strassen(a, b):
    assert a.shape[0] == a.shape[1] and a.shape[0] == get_deg(a.shape[0])
    assert b.shape[0] == b.shape[1] and b.shape[0] == get_deg(b.shape[0])
    assert a.shape[0] == b.shape[0]
    size = int(a.shape[0]/2)
    if size <= 8:
        return dot_n3(a, b)
    
    # a submatrixes
    a11 = a[:size, :size]
    a12 = a[:size, size:]
    a21 = a[size:, :size]
    a22 = a[size:, size:]
#     print(a11, a12, a21, a22, sep='\n')
    
    # b submatrixes
    b11 = b[:size, :size]
    b12 = b[:size, size:]
    b21 = b[size:, :size]
    b22 = b[size:, size:]
#     print(b11, b12, b21, b22, sep='\n')

    # help matrix
    p1 = strassen(a11 + a22, b11 + b22)
    p2 = strassen(a21 + a22, b11)
    p3 = strassen(a11, b12 - b22)
    p4 = strassen(a22, b21 - b11)
    p5 = strassen(a11 + a12, b22)
    p6 = strassen(a21 - a11, b11 + b12)
    p7 = strassen(a12 - a22, b21 + b22)
    
    # answer
    c11 = p1 + p4 - p5 + p7
    c12 = p3 + p5
    c21 = p2 + p4
    c22 = p1 - p2 + p3 + p6
    
    c = np.concatenate([
        np.concatenate([c11, c12], axis=1),
        np.concatenate([c21, c22], axis=1)
    ], axis=0)
    return c


def strip_matrix(a, size):
    return a[:size, :size]


def matix2Z(a, Z=9):
    shape = a.shape
    return np.array(list(map(
        lambda x: x%Z, a
    )), dtype=int).reshape(shape)


def fast_matrix_deg(a, n):
    if n == 0:
        return np.eye(a.shape[0])
    if n == 1:
        return a
    
    if n % 2 == 0:
        return fast_matrix_deg(strassen(a, a), int(n/2))
    else:
        return strassen(a, fast_matrix_deg(strassen(a, a), int((n - 1)/2)))


def read_matrix():
    a = list(map(int, stdin.readline().rstrip().split()))
    n = len(a)
    for _ in range(n - 1):
        a.extend(list(map(int, stdin.readline().rstrip().split())))
        
    return np.array(a, dtype=int).reshape((n, n))


def print_matrix(a):
    for line in a:
        print(*line, sep=' ')


def do_task(a=None):
    if a is None:
        a = read_matrix()
    assert a.shape[0] == a.shape[1]
    n = a.shape[0]
    a_wide = size2deg(a)
    a_deg = fast_matrix_deg(a_wide, n)
    a_strip = strip_matrix(a_deg, n)
    a_z9 = matix2Z(a_strip, Z=9)
    return a_z9


a = do_task()
print_matrix(a)

/Users/maxim/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


In [165]:
with open('a.txt', 'r') as f:
    a = list(map(int, f.readline().rstrip().split()))
    n = len(a)
    for _ in range(n - 1):
        a.extend(list(map(int, f.readline().rstrip().split())))
        
    a = np.array(a, dtype=int).reshape((n, n))

In [166]:
a

array([[0, 1, 1, 2, 3, 4, 5, 7, 7, 4],
       [3, 6, 0, 5, 7, 4, 3, 1, 6, 7],
       [7, 5, 5, 4, 7, 4, 3, 4, 5, 7],
       [4, 4, 7, 7, 5, 4, 3, 3, 2, 1],
       [4, 5, 0, 1, 6, 7, 0, 5, 1, 3],
       [2, 2, 4, 0, 4, 5, 2, 5, 7, 0],
       [3, 5, 1, 3, 4, 3, 7, 6, 3, 6],
       [6, 2, 4, 7, 3, 5, 1, 3, 0, 0],
       [4, 5, 0, 5, 1, 3, 0, 6, 2, 4],
       [3, 4, 0, 5, 6, 5, 4, 4, 3, 6]])

In [167]:
size = a.shape[0]
aa = do_task(a)

res = a
for i in range(size-1):
    res = np.dot(res, a)
res = matix2Z(res, Z=9)

print(np.array_equal(aa, res))

True


In [168]:
print_matrix(a)
print()
print_matrix(aa)
print()
print_matrix(res)

0 1 1 2 3 4 5 7 7 4
3 6 0 5 7 4 3 1 6 7
7 5 5 4 7 4 3 4 5 7
4 4 7 7 5 4 3 3 2 1
4 5 0 1 6 7 0 5 1 3
2 2 4 0 4 5 2 5 7 0
3 5 1 3 4 3 7 6 3 6
6 2 4 7 3 5 1 3 0 0
4 5 0 5 1 3 0 6 2 4
3 4 0 5 6 5 4 4 3 6

0 8 8 1 6 4 8 1 8 7
6 5 3 7 0 6 6 4 2 1
5 7 7 2 1 0 5 0 4 6
8 1 6 7 7 6 6 0 1 3
3 7 0 6 1 8 2 7 0 8
0 8 0 0 8 5 4 4 7 6
1 8 7 7 8 4 7 5 3 1
6 8 1 5 5 4 8 7 4 0
0 1 1 3 7 4 2 0 8 4
2 4 1 8 2 2 3 6 8 6

0 8 8 1 6 4 8 1 8 7
6 5 3 7 0 6 6 4 2 1
5 7 7 2 1 0 5 0 4 6
8 1 6 7 7 6 6 0 1 3
3 7 0 6 1 8 2 7 0 8
0 8 0 0 8 5 4 4 7 6
1 8 7 7 8 4 7 5 3 1
6 8 1 5 5 4 8 7 4 0
0 1 1 3 7 4 2 0 8 4
2 4 1 8 2 2 3 6 8 6


In [156]:
%%time
size = 10
a = np.random.randint(0, 8, size=size*size).reshape((size, size))
aa = do_task(a)

res = a
for i in range(size-1):
    res = np.dot(res, a)
res = matix2Z(res, Z=9)

print(np.array_equal(aa, res))

True
CPU times: user 12.4 ms, sys: 1.35 ms, total: 13.8 ms
Wall time: 13.3 ms


In [157]:
print_matrix(a)
print()
print_matrix(aa)
print()
print_matrix(res)

0 1 1 2 3 4 5 7 7 4
3 6 0 5 7 4 3 1 6 7
7 5 5 4 7 4 3 4 5 7
4 4 7 7 5 4 3 3 2 1
4 5 0 1 6 7 0 5 1 3
2 2 4 0 4 5 2 5 7 0
3 5 1 3 4 3 7 6 3 6
6 2 4 7 3 5 1 3 0 0
4 5 0 5 1 3 0 6 2 4
3 4 0 5 6 5 4 4 3 6

0 8 8 1 6 4 8 1 8 7
6 5 3 7 0 6 6 4 2 1
5 7 7 2 1 0 5 0 4 6
8 1 6 7 7 6 6 0 1 3
3 7 0 6 1 8 2 7 0 8
0 8 0 0 8 5 4 4 7 6
1 8 7 7 8 4 7 5 3 1
6 8 1 5 5 4 8 7 4 0
0 1 1 3 7 4 2 0 8 4
2 4 1 8 2 2 3 6 8 6

0 8 8 1 6 4 8 1 8 7
6 5 3 7 0 6 6 4 2 1
5 7 7 2 1 0 5 0 4 6
8 1 6 7 7 6 6 0 1 3
3 7 0 6 1 8 2 7 0 8
0 8 0 0 8 5 4 4 7 6
1 8 7 7 8 4 7 5 3 1
6 8 1 5 5 4 8 7 4 0
0 1 1 3 7 4 2 0 8 4
2 4 1 8 2 2 3 6 8 6


In [77]:
np.array_equal(strassen(aa, bb), np.dot(aa, bb))

True